In [1]:
import boto3
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
import numpy as np
import io
import os 

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split, Subset, ConcatDataset
from torchvision.transforms import transforms
import numpy as np
from torchvision import models, datasets
from torchsummary import summary

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

### Load Data

In [6]:
class LocalData:
    def __init__ (self, clinic_id): #data_range will be removed for final code
        self.clinic_id = clinic_id
        self.path = f'../120_dataset/{clinic_id}/'
        #self.range = data_range #remove for final code
        
        
    def dataset (self):
        transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to a fixed size (optional)
        transforms.ToTensor(),          # Convert images to PyTorch tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]) 
        ])

# Load the dataset from the train folder
        train_dataset = datasets.ImageFolder(root=f'{self.path}', transform=transform)
        #subset_indices = list(self.range)    #remove for final code
        #train_dataset = Subset (train_dataset, subset_indices) #remove for final code

        train_size = int(0.8*len(train_dataset))
        val_size = len(train_dataset)-train_size

        train_subset, val_subset = random_split(train_dataset, [train_size, val_size])
        return train_subset, val_subset
    
    def dataloader(self):  
        train_subset, val_subset = self.dataset()
        train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

        print (f'loading {self.clinic_id}')
        return train_loader, val_loader

### Models

In [7]:
# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def resnet18 (weights='DEFAULT'): #James
    
    resnet18 = models.resnet18(weights = weights).to(device)
    for param in resnet18.parameters():
        param.requires_grad = False
    resnet18.fc = nn.Sequential (
    nn.Linear(in_features = 512, out_features = 256, bias = True),
    nn.Dropout(p = 0.5),
    nn.Linear(in_features = 256, out_features = 1, bias = True),
    nn.Sigmoid()
    )
    
    for param in resnet18.fc.parameters():
        param.requires_grad = True
        
    resnet18.__class__.__name__ = 'ResNet18'
    return resnet18

In [8]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, drop_out=0.2):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.drop_out = nn.Dropout(drop_out)
        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.drop_out(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            out += self.downsample(x)
        out = self.relu(out)
        out = self.drop_out(out)
        return out

class CustomResNet18(nn.Module):
    def __init__(self, drop_out=0.2):
        super(CustomResNet18, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Replace residual blocks with custom BasicBlock including dropout
        self.layer1 = self._make_layer(BasicBlock, 64, 2, stride=1, drop_out=drop_out)
        self.layer2 = self._make_layer(BasicBlock, 128, 2, stride=2, drop_out=drop_out)
        self.layer3 = self._make_layer(BasicBlock, 256, 2, stride=2, drop_out=drop_out)
        self.layer4 = self._make_layer(BasicBlock, 512, 2, stride=2, drop_out=drop_out)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, 1000)


    def _make_layer(self, block, out_channels, num_blocks, stride, drop_out):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride, drop_out))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def custom_resnet18(weights='DEFAULT', drop_out=0.2):
    custom_resnet18 = CustomResNet18(drop_out=0.2)
    custom_resnet18.load_state_dict(models.resnet18(weights = weights).state_dict(), strict=False)

# Then freeze parameters
    for param in custom_resnet18.parameters():
        param.requires_grad = False

    custom_resnet18.fc = nn.Sequential (
    nn.Linear(in_features = 512, out_features =1),
    nn.Sigmoid()
    )
    for param in custom_resnet18.fc.parameters():
        param.requires_grad = True
    return custom_resnet18



In [9]:
def vgg16(weights = 'DEFAULT'): #David
    vgg16 = models.vgg16(weights=weights).to(device)

# Freeze the parameters of the base model
    for param in vgg16.features.parameters():
        param.requires_grad = False
    
# Modify the classifier part for binary classification
    vgg16.classifier[6] = nn.Sequential(
        nn.Linear(vgg16.classifier[6].in_features, 512),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(512, 1),
        nn.Sigmoid()
    )
    
    vgg16.__class__.__name__ = 'VGG16'
    return vgg16

    
def vgg19 (weights='DEFAULT'):
    vgg19 = models.vgg19 (weights=weights).to(device)
    
    for param in vgg19.parameters():
        param.requires_grad = False
        
    vgg19.classifier = nn.Sequential (
        nn.Linear(25088, 4096),        
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 4096),       
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 1),
        nn.Sigmoid()          
    )
    for param in vgg19.classifier.parameters():
        param.requires_grad = True
    
    vgg19.__class__.__name__ = 'VGG19'
    return vgg19

### Training

In [10]:
import torch
import torch.optim as optim


# Define the device - mps (Metal) for macOS GPU, otherwise default to 'cuda' or 'cpu'
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

def calculate_accuracy(outputs, labels, threshold=0.5):
    preds = (outputs > threshold).float()
    correct = (preds == labels).float().sum()
    accuracy = correct / labels.size(0)
    return accuracy

def train_local_model(model, train_loader, val_loader, num_epochs=10):
    model = model.to(device)

    criterion = torch.nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        running_accuracy = 0.0
        total_train = 0
        threshold = 0.5

        for images, labels in train_loader:
            optimizer.zero_grad()
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).squeeze(1)
            # Calculate loss
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_accuracy += calculate_accuracy(outputs, labels, threshold)
            total_train += 1


        # Validation phase
        model.eval()
        val_loss = 0.0
        val_accuracy = 0.0
        total_val = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images).squeeze(1)
                loss = criterion(outputs, labels.float())
                val_loss += loss.item()
                val_accuracy += calculate_accuracy(outputs, labels, threshold)
                total_val += 1

        avg_val_loss = val_loss / total_val
        avg_val_accuracy = val_accuracy / total_val
        print (f"Epoch {epoch + 1}/{num_epochs}:\ntrain_loss: {running_loss / total_train}, train_accuracy: {running_accuracy / total_train}\nValidation Loss: {avg_val_loss:.4f}, Validation Accuracy: {avg_val_accuracy:.4f}")
    
    return model, avg_val_loss, avg_val_accuracy


### Federated Learning

In [11]:
def federated_averaging (client_weights):
    avg_weights = client_weights[0].copy()
    
    for key in avg_weights.keys():
        for key in avg_weights.keys():
            for i in range (1, len (client_weights)):
                avg_weights[key] += client_weights[i][key]
                
            avg_weights[key] = avg_weights[key] / len (client_weights)
            
    return avg_weights

In [12]:
def federated_learning (model, num_clients, num_rounds, train_loaders, val_loaders):
    global_model = model('DEFAULT')
    global_weights = global_model.state_dict()
    
    for round_num in range (num_rounds):
        print (f'Round {round_num+1}')
        
        client_weights = []
        
        for client_id in range (num_clients):
            print (f'client {client_id+1} training...')
            
            local_model = model(None)
            local_model.load_state_dict (global_weights)
            local_model.to(device)
            
            client_train_loader = train_loaders[client_id]
            client_val_loader = val_loaders[client_id]
            
            output_model, _, _ = train_local_model (local_model, client_train_loader, client_val_loader)
            client_updated_weights = output_model.state_dict()
            
            client_weights.append (client_updated_weights)
            
        global_weights = federated_averaging (client_weights)
        
        global_model.load_state_dict (global_weights)
    return global_model

### Experimental Setup

### Evaluation

In [13]:
#Setting up dataset for training the FL model
num_clients = 4

num_rounds = 3

train_loader_0, val_loader_0 = LocalData('clinic_0').dataloader() #replace range() for testing, and remove when the code is ready for the final run.

train_loader_1, val_loader_1 = LocalData('clinic_1').dataloader() #replace range() for testing, and remove when the code is ready for the final run.

train_loader_2, val_loader_2 = LocalData('clinic_2').dataloader() #replace range() for testing, and remove when the code is ready for the final run.

train_loader_3, val_loader_3 = LocalData('clinic_3').dataloader() #replace range() for testing, and remove when the code is ready for the final run.

train_loaders = [train_loader_0, train_loader_1, train_loader_2, train_loader_3]
val_loaders = [val_loader_0, val_loader_1, val_loader_2, val_loader_3]

loading clinic_0
loading clinic_1
loading clinic_2
loading clinic_3


In [14]:
# Metrics
def metrics (ground_truths, predictions):
    accuracy = round(accuracy_score(ground_truths, predictions), 4)
    precision = round(precision_score(ground_truths, predictions), 4)
    recall = round(recall_score(ground_truths, predictions), 4)
    f1 = round(f1_score(ground_truths, predictions), 4)
    confusion_ma = confusion_matrix (ground_truths, predictions)
    
    print ('Accuracy score: ', accuracy)
    print ('Precision score: ', precision)
    print ('Recall score: ', recall)
    print ('F1 score: ', f1)
    print ('Confusion Matrix: \n', confusion_ma)
    return accuracy, precision, recall, f1
    

In [15]:
#Train client model on clinic4's train_loader
#Make prediction on clinic4's val_loader
def evaluation(client_model):
    client_model.eval()
    predictions = []
    ground_truths = []

    with torch.no_grad():
        for images, labels in val_loader_clinic5:
            images, labels = images.to(device), labels.to(device)
            output = client_model(images)
            #output = output.round()
            predictions.append (output.cpu())
            ground_truths.append (labels.cpu())

    predictions = np.concatenate (predictions).reshape(-1).astype ('int')
    ground_truths = np.concatenate (ground_truths)
    
    return metrics (ground_truths, predictions)

In [16]:
results = []
client_models = [custom_resnet18, resnet18, vgg16, vgg19]

#### 1. Evaluation on the held-out clinic
After several rounds of training, the global model's weights are now used as initiallized weights for a fresh client model. Then, we will use this model to make prediction on the eval dataset on the 5th clinic's data.

In [17]:
#Setting up dataset for evaluation on clinic 5
train_loader_clinic5, val_loader_clinic5 = LocalData ('clinic_4').dataloader()

loading clinic_4


In [18]:
for client_model in client_models:
    model_name = client_model().__class__.__name__
    client_model = train_local_model (client_model('DEFAULT'), train_loader_clinic5, val_loader_clinic5)[0]
    accuracy, precision, recall, f1 = evaluation (client_model)
    results.append ([model_name, accuracy, precision, recall, f1])

Epoch 1/10:
train_loss: 0.6689575953951364, train_accuracy: 0.6005431413650513
Validation Loss: 0.6894, Validation Accuracy: 0.5765
Epoch 2/10:
train_loss: 0.661827098531059, train_accuracy: 0.6045064330101013
Validation Loss: 0.6970, Validation Accuracy: 0.5150
Epoch 3/10:
train_loss: 0.655998528192315, train_accuracy: 0.6089033484458923
Validation Loss: 0.6997, Validation Accuracy: 0.5019
Epoch 4/10:
train_loss: 0.6488239944358415, train_accuracy: 0.617552638053894
Validation Loss: 0.6997, Validation Accuracy: 0.5051
Epoch 5/10:
train_loss: 0.6461738672437547, train_accuracy: 0.6128057837486267
Validation Loss: 0.7117, Validation Accuracy: 0.4525
Epoch 6/10:
train_loss: 0.6430067810453947, train_accuracy: 0.6196293830871582
Validation Loss: 0.7154, Validation Accuracy: 0.4326
Epoch 7/10:
train_loss: 0.6408312579121771, train_accuracy: 0.6237828731536865
Validation Loss: 0.7281, Validation Accuracy: 0.4164
Epoch 8/10:
train_loss: 0.6364408348557316, train_accuracy: 0.6259508728981018


/Users/davidbain/Library/Caches/pypoetry/virtualenvs/ilab-YBgqLwdb-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy score:  0.6149
Precision score:  0.0
Recall score:  0.0
F1 score:  0.0
Confusion Matrix: 
 [[1552    0]
 [ 972    0]]
Epoch 1/10:
train_loss: 0.45087515482608276, train_accuracy: 0.7904089689254761
Validation Loss: 0.3895, Validation Accuracy: 0.8284
Epoch 2/10:
train_loss: 0.34992549766468095, train_accuracy: 0.8478654026985168
Validation Loss: 0.3488, Validation Accuracy: 0.8463
Epoch 3/10:
train_loss: 0.33216391965935504, train_accuracy: 0.8565679788589478
Validation Loss: 0.3155, Validation Accuracy: 0.8626
Epoch 4/10:
train_loss: 0.32270526895417445, train_accuracy: 0.8629883527755737
Validation Loss: 0.3116, Validation Accuracy: 0.8629
Epoch 5/10:
train_loss: 0.3110676409700249, train_accuracy: 0.868579626083374
Validation Loss: 0.3115, Validation Accuracy: 0.8638
Epoch 6/10:
train_loss: 0.30638605360931986, train_accuracy: 0.8700706362724304
Validation Loss: 0.3187, Validation Accuracy: 0.8630
Epoch 7/10:
train_loss: 0.3010253634869675, train_accuracy: 0.875250995159149

/Users/davidbain/Library/Caches/pypoetry/virtualenvs/ilab-YBgqLwdb-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy score:  0.6149
Precision score:  0.0
Recall score:  0.0
F1 score:  0.0
Confusion Matrix: 
 [[1552    0]
 [ 972    0]]
Epoch 1/10:
train_loss: 0.38672950227237957, train_accuracy: 0.8210123777389526
Validation Loss: 0.3326, Validation Accuracy: 0.8591
Epoch 2/10:
train_loss: 0.2532092814769926, train_accuracy: 0.8944894075393677
Validation Loss: 0.3857, Validation Accuracy: 0.8445
Epoch 3/10:
train_loss: 0.1654463526380213, train_accuracy: 0.929641842842102
Validation Loss: 0.4365, Validation Accuracy: 0.8571
Epoch 4/10:
train_loss: 0.09738125811077512, train_accuracy: 0.9623752236366272
Validation Loss: 0.4676, Validation Accuracy: 0.8631
Epoch 5/10:
train_loss: 0.06704093699853372, train_accuracy: 0.9761669039726257
Validation Loss: 0.6019, Validation Accuracy: 0.8564
Epoch 6/10:
train_loss: 0.05350572473179443, train_accuracy: 0.9792326092720032
Validation Loss: 0.7607, Validation Accuracy: 0.8536
Epoch 7/10:
train_loss: 0.04390847612808965, train_accuracy: 0.983682751655578

In [ ]:
for client_model in client_models:
    global_model = federated_learning (client_model, num_clients, 2, train_loaders, val_loaders) #change num_rounds according to your GPU, which mine doesn't have one :<.
    model = train_local_model (global_model, train_loader_clinic5, val_loader_clinic5)[0]
    accuracy, precision, recall, f1 = evaluation (model)
    model_name = f'FedAVG {client_model().__class__.__name__}'
    results.append ([model_name, accuracy, precision, recall, f1])

Round 1
client 1 training...
Epoch 1/10:
train_loss: 0.6521381333516478, train_accuracy: 0.6353521943092346
Validation Loss: 0.6375, Validation Accuracy: 0.6345
Epoch 2/10:
train_loss: 0.6427800775312418, train_accuracy: 0.6351586580276489
Validation Loss: 0.6425, Validation Accuracy: 0.6245
Epoch 3/10:
train_loss: 0.6398949375831675, train_accuracy: 0.6352553963661194
Validation Loss: 0.6536, Validation Accuracy: 0.6109
Epoch 4/10:
train_loss: 0.6360555457256896, train_accuracy: 0.6353521943092346
Validation Loss: 0.6761, Validation Accuracy: 0.5693
Epoch 5/10:
train_loss: 0.6333574067697436, train_accuracy: 0.6369001269340515
Validation Loss: 0.6788, Validation Accuracy: 0.5636
Epoch 6/10:
train_loss: 0.6297193536817474, train_accuracy: 0.6372871398925781
Validation Loss: 0.6722, Validation Accuracy: 0.5857
Epoch 7/10:
train_loss: 0.6290167071870975, train_accuracy: 0.6407701373100281
Validation Loss: 0.6827, Validation Accuracy: 0.5511
Epoch 8/10:
train_loss: 0.623882659540826, trai

In [ ]:
df_eval = pd.DataFrame (columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1'],
                       data = results)

In [ ]:
df_eval

In [ ]:
# client_models[0]

#### 2. Evaluation on each client's validation data

In [ ]:
results_2 = []

In [ ]:

for client_model in client_models:
    model_name = client_model().__class__.__name__
    for i in range (4):
        local_model = train_local_model (client_model('DEFAULT'), train_loaders[i], val_loaders[i])[0]
        accuracy, precision, recall, f1 = evaluation (local_model)
        results_2.append ([model_name, f'Clinic_{i}', accuracy, precision, recall, f1])

In [ ]:
for client_model in client_models:
    global_model = federated_learning (client_model, num_clients, 2, train_loaders, val_loaders) #change num_rounds according to your GPU, which mine doesn't have one :<.
    for i in range (4):
        model = train_local_model (global_model, train_loaders[i], val_loaders[i])[0]
        accuracy, precision, recall, f1 = evaluation (model)
        model_name = f'FedAVG {client_model().__class__.__name__}'
        results_2.append ([model_name, f'Clinic_{i}', accuracy, precision, recall, f1])

In [ ]:
df_eval_2 = pd.DataFrame (columns = ['Model', 'Clinic', 'Accuracy', 'Precision', 'Recall', 'F1'],
                         data = results_2)

In [ ]:
resnet18()

In [ ]:
a = [1, 2, 3]
b = [4, 5, 6]

for c, d in zip (a, b):
    print (c, d)